# Линейная регрессия
## Одномерный случай

Имеем линейную модель $a(x) = w * x + b$, тогда функция потерь с MSE будет следующей: $L = 1 / N * \sum_{i=1}^{N} (y_i - wx_i - b)^2$

Производная по w будет следующей: $\frac{\partial L}{\partial w} = -2 / N * \sum_1^N x_i(y_i - wx_i - b)$

А производная по b: $\frac{\partial L}{\partial b} = - 2 / N * \sum_1^N (y_i - wx_i - b)$

## Многомерный случай

Путь $x_i$ - вектор, такой что $x_i = (x_{i, 1}, x_{i, 2}, ..., x_{i, M})$, тогда модель принимает вид $a(x) = (w, x) + b$, где $(w, x)$ - скалярное произведение векторов.

Тогда $L = 1/N * \sum_1^N (y_i - (w, x_i) - b)^2$

Частные производные по параметрам:

$\frac{\partial L}{\partial b} = -2/N * \sum_1^N (y_i - (w, x_i) - b)$

$\frac{\partial L}{\partial w_1} = -2/N * \sum_1^N x_{i,1}(y_i - (w, x_i) - b)$

$\frac{\partial L}{\partial w_M} = -2/N * \sum_1^N x_{i,M}(y_i - (w, x_i) - b)$

## Укорение вычислений

Предсохраним вычисления для $(y_i - (w, x_i) - b)$